In [17]:
import torch
import torchvision
import torch.nn as nn

# Load MNIST data

In [67]:
# Calling Train set.
#
train = torchvision.datasets.MNIST(root='MNIST/processed', train=True, download=True, transform=torchvision.transforms.ToTensor())

## Split 60K to 50K and 10K for Train and Val sets.
# Followed by applying loaders.
#
train_dl, valid_dl = torch.utils.data.random_split(train, [50000, 10000])
trainloader = torch.utils.data.DataLoader(train_dl, batch_size=16, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valid_dl, batch_size=16, shuffle=True, num_workers=2)

# Calling Test set.
#
test = torchvision.datasets.MNIST(root='MNIST/processed', train=False, download=True, transform=torchvision.transforms.ToTensor())
testloader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=True, num_workers=2)

In [68]:
# Instance-label split for Train and Test.
#
train_instances = torch.cat([instance for instance, label in trainloader])
train_labels = torch.cat([label for instance, label in trainloader])

test_instances = torch.cat([instance for instance, label in testloader])
test_labels = torch.cat([label for instance, label in testloader])

# CNN Architecture Class

In [69]:
class CnnP2(nn.Module):
    """
    Convolutional Neural Network Architecture based on the description. 
    * Layers initialization.
    * Forward() function.
    """

    def __init__(self):
        super(CnnP2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(2)
        self.linear = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [70]:
def main():
    
    model = CnnP2()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(2):  # loop over the dataset multiple times

        correct = 0.0
        total = 0.0

        # for i, data in enumerate(trainloader, 0):
        # for i, data in enumerate(trainloader, 0):
        for i, data in enumerate(valloader, 0):
            
            inputs, labels = data
            optimizer.zero_grad()

            # forward + backward + optimize
            #
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f'[Iter {epoch+1}] - Accuracy: {100 * correct // total} %')

    print('Finished Training!')

In [71]:
if __name__ == "__main__":
    main()

[Iter 1] - Accuracy: 93.0 %
[Iter 2] - Accuracy: 100.0 %
Finished Training!
